In [1]:
import pandas as pd 

In [2]:
import numpy as np # library to handle data in a vectorized manner

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/osx-64::statsmodels==0.8.0=py36h9c68fc9_0
  - defaults/osx-64::patsy==0.4.1=py36ha1b3fa5_0
  - defaults/osx-64::numba==0.35.0=np113py36_6
  - defaults/osx-64::scikit-image==0.13.0=py36h398857d_1
  - defaults/osx-64::blaze==0.11.3=py36h02e7a37_0
  - defaults/osx-64::scikit-learn==0.19.1=py36hffbff8c_0
  - defaults/osx-64::numexpr==2.6.2=py36h0f4f1da_1
  - defaults/osx-64::scipy==0.19.1=py36h3e758e1_3
  - defaults/osx-64::anaconda==5.0.1=py36h6e48e2d_1
  - defaults/osx-64::pytables==3.4.2=py36hfbd7ab0_2
  - defaults/osx-64::seaborn==0.8.0=py36h74df97e_0
done

## Package Plan ##

  environment location: /Users/parul/anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.

In [4]:
import requests # library to handle requests

from bs4 import BeautifulSoup

In [5]:
url=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(url,'lxml')
my_table = soup.find('table',{'class':'wikitable sortable'})
neigh=my_table.findAll('tr')

In [6]:
neigh

[<tr>
 <th>Postcode</th>
 <th>Borough</th>
 <th>Neighbourhood
 </th></tr>, <tr>
 <td>M1A</td>
 <td>Not assigned</td>
 <td>Not assigned
 </td></tr>, <tr>
 <td>M2A</td>
 <td>Not assigned</td>
 <td>Not assigned
 </td></tr>, <tr>
 <td>M3A</td>
 <td><a href="/wiki/North_York" title="North York">North York</a></td>
 <td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
 </td></tr>, <tr>
 <td>M4A</td>
 <td><a href="/wiki/North_York" title="North York">North York</a></td>
 <td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
 </td></tr>, <tr>
 <td>M5A</td>
 <td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
 <td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
 </td></tr>, <tr>
 <td>M5A</td>
 <td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
 <td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
 </td></tr>, <tr>
 <td>M6A</td>
 <td

<h5> Create DataFrame</h5>

In [7]:
# define the dataframe columns
column_names = ['PostalCode','Borough','Neighborhood'] 

# instantiate the dataframe
can_neighborhoods = pd.DataFrame(columns=column_names)

In [8]:
can_neighborhoods

,PostalCode,Borough,Neighborhood


In [9]:
# Parse the table into a Pandas dataframe
i=0
for i in range(len(neigh)):
    if (neigh[i].contents[3].text != 'Not assigned' or neigh[i].contents[5].text != 'Not assigned\n'):
        can_neighborhoods.loc[i]=[neigh[i].contents[1].text,neigh[i].contents[3].text,neigh[i].contents[5].text]
        i=i+1
can_neighborhoods['Neighborhood'] = can_neighborhoods['Neighborhood'].str.replace('\n','')
can_neighborhoods.drop(can_neighborhoods[can_neighborhoods.Borough == 'Not Assigned'].index,inplace=True)
can_neighborhoods.drop(can_neighborhoods[can_neighborhoods.PostalCode == 'Postcode'].index,inplace=True)

for i in range(len(can_neighborhoods)):
    if can_neighborhoods.iloc[i]['Neighborhood'] == 'Not assigned':
        can_neighborhoods.iloc[i]['Neighborhood'] = can_neighborhoods.iloc[i]['Borough']

In [10]:
can_neighborhoods.head(10)

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Queen's Park
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern


<h5>Cleaning Data - Group by Postal Code</h5>

In [12]:
newgroup=can_neighborhoods.groupby(['PostalCode','Borough'])
final_df=pd.DataFrame(columns=column_names)

In [13]:
i=0
for name,group in newgroup:
    final_df.loc[i]=[list(name)[0],list(name)[1],group['Neighborhood'].tolist()]
    i=i+1
final_df['Neighborhood'] = final_df['Neighborhood'].apply(lambda x: ','.join(map(str, x)))

In [14]:
final_df.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


<h5>print the number of rows of your dataframe</h5>

In [15]:
final_df.shape

(103, 3)

<h5>Loading geographical coordinates, loading file</h5>

In [17]:
# Read csv file with 

geocord=pd.read_csv('http://cocl.us/Geospatial_data')

In [18]:
geocord

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [19]:
for i in range(len(geocord)):
    final_df.at[final_df['PostalCode']==geocord.iloc[i]['Postal Code'],'Latitude']=geocord.iloc[i]['Latitude']
    final_df.at[final_df['PostalCode']==geocord.iloc[i]['Postal Code'],'Longitude']=geocord.iloc[i]['Longitude']

In [28]:
final_df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


In [21]:
final_df.shape

(103, 5)

<h5>Explore and cluster the neighborhoods in Toronto</h5>

In [22]:
import folium
from geopy.geocoders import Nominatim

In [23]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


<h5>Create a map of Toronto with neighborhoods superimposed on top.</h5>

In [25]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(final_df['Latitude'], final_df['Longitude'], final_df['Borough'], final_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<h5>Define Foursquare Credentials and Version</h5>

In [26]:
CLIENT_ID = '5ABHQPNPEU02QZWSUE0ASEOTIZYKOQJ1LYHTWETQCHGUKOQN' # your Foursquare ID
CLIENT_SECRET = 'V2SB2PGE1RWNW0GVT5N1NOFMB5AUZJKTESWMB4VY0GGK5AR3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5ABHQPNPEU02QZWSUE0ASEOTIZYKOQJ1LYHTWETQCHGUKOQN
CLIENT_SECRET:V2SB2PGE1RWNW0GVT5N1NOFMB5AUZJKTESWMB4VY0GGK5AR3


In [31]:
# select a neighborhood 

final_df[final_df.Neighborhood=="Roselawn"]

,PostalCode,Borough,Neighborhood,Latitude,Longitude
63,M5N,Central Toronto,Roselawn,43.711695,-79.416936


<h5>Get the neighborhood's latitude and longitude values.</h5>

In [32]:
neighborhood_latitude = final_df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = final_df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = final_df.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge,Malvern are 43.806686299999996, -79.19435340000001.


<h5>Get the top 100 venues that are in Roselawn within a radius of 500 meters</h5>

In [33]:

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius


url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=5ABHQPNPEU02QZWSUE0ASEOTIZYKOQJ1LYHTWETQCHGUKOQN&client_secret=V2SB2PGE1RWNW0GVT5N1NOFMB5AUZJKTESWMB4VY0GGK5AR3&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100'

In [34]:
results = requests.get(url).json()

In [35]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

<h5>clean the json and structure it into a pandas dataframe.</h5>

In [36]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.807448,-79.199056


And how many venues were returned by Foursquare?

In [37]:

print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


<h5>Explore all Neighborhoods in Toronto</h5>

In [38]:

# First create a function to retrieve venues from all Neighborhoods.

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [42]:
# We apply the previous function to the Toronto dataframe

toronto_venues = getNearbyVenues(names=final_df['Neighborhood'],
                                   latitudes=final_df['Latitude'],
                                   longitudes=final_df['Longitude'])

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beaches West,Indi

In [43]:
# Size of the resulting dataframe with venues in each Neighborhood of toronto

print(toronto_venues.shape)
toronto_venues.head()

(2245, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
3,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [44]:
#How many values were returned for each Neighborhood?

toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",2,2,2,2,2,2
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",12,12,12,12,12,12
"Alderwood,Long Branch",10,10,10,10,10,10
"Bathurst Manor,Downsview North,Wilson Heights",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",24,24,24,24,24,24
Berczy Park,55,55,55,55,55,55


<h5>Unique categories can be curated from all the returned venues?</h5>

In [45]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 275 uniques categories.
